# Dados - Q4

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import urllib

## API
Dados obtidos em: 
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202109/

Data escolhida: (10/09/2021)

In [2]:
import urllib
url = 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/202109/oisst-avhrr-v02r01.20210910.nc'
temp_nc = "oisst-avhrr-v02r01.20210910.nc"
urllib.request.urlretrieve(url, temp_nc)

('oisst-avhrr-v02r01.20210910.nc', <http.client.HTTPMessage at 0x7fbbe40c6bb0>)

## Recortes e cálculos

In [123]:
def longitude_formatter(val=float):
    '''
    Adapta longitude de -180 > 180 para 0 > 360
    
    Valores entre 0 e 180 são graus a Leste, estes se mantém.
    Enquanto se forem entre -180 e 0, soma-se 360.

    **Args:
    ----------
    val: float


    Returns:
    -------
    obj: <float>
    '''
    if val < 0:
        val += 360
    return val

In [126]:
def dataset_reader(path, lat_slice=None, lon_slice=None):
    '''
    Lê o dado netcdf e recorta lat e lon.
    
    **Args:
    ----------
    path: str
    lat_slice: list
    lon_slice: list

    Returns:
    -------
    obj: xarray.Dataset
    '''
    sst = xr.open_dataset(path)

    if lat_slice != None:
        sst = sst.sel(lat=slice(lat_slice[0], lat_slice[1]))
        
    if lon_slice != None:
        # deve entrar o <longitude_formatter()> pra adaptar os valores de entrada em lon_slice pros padrões do dataset (0-360) 
        lon_slice[0] = longitude_formatter(lon_slice[0])
        lon_slice[1] = longitude_formatter(lon_slice[1])

        sst = sst.sel(lon=slice(lon_slice[0], lon_slice[1]))
        
    return sst

In [156]:
Nino1_2 = dataset_reader("oisst-avhrr-v02r01.20210910.nc", lat_slice=[-10,0], lon_slice=[-90,-80])
Nino3_4 = dataset_reader("oisst-avhrr-v02r01.20210910.nc", lat_slice=[-5,5], lon_slice=[-170,-120])
cbm = dataset_reader("oisst-avhrr-v02r01.20210910.nc", lat_slice=[-43,-33], lon_slice=[-63,-48])
positions = {'Nino 1+2':Nino1_2, 'Nino 3_4':Nino3_4, 'CBM':cbm}

In [157]:
# Calculando as médias
for cada in positions:
    media = positions[cada].sst.mean().values
    positions[cada] = media

In [180]:
# Transformando em dataframe
df = pd.DataFrame(index=positions.keys())
df['SST'] = positions.values()

In [181]:
# salvando o arquivo
df.to_csv('sea_surface_temperature.csv')
df

,SST
Nino 1+2,20.662952
Nino 3_4,26.588938
CBM,12.779157
